In [1]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [7]:
agri_data = pd.read_csv("C:\Users\CJHx6\Downloads\Production_Crops_Livestock_E_All_Data\Production_Crops_Livestock_E_All_Data.csv")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1380613774.py, line 1)

In [9]:
agri_data = pd.read_csv(r"C:\Users\CJHx6\Downloads\Production_Crops_Livestock_E_All_Data\Production_Crops_Livestock_E_All_Data.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 84095: invalid continuation byte

In [13]:
agri_data = pd.read_csv(r"C:\Users\CJHx6\Downloads\Production_Crops_Livestock_E_All_Data\Production_Crops_Livestock_E_All_Data.csv", encoding='ISO-8859-1', low_memory = False)


In [15]:
agri_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79606 entries, 0 to 79605
Columns: 195 entries, Area Code to Y2022N
dtypes: float64(62), int64(3), object(130)
memory usage: 118.4+ MB


In [21]:
agri_data.columns

Index(['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item Code (CPC)',
       'Item', 'Element Code', 'Element', 'Unit', 'Y1961',
       ...
       'Y2019N', 'Y2020', 'Y2020F', 'Y2020N', 'Y2021', 'Y2021F', 'Y2021N',
       'Y2022', 'Y2022F', 'Y2022N'],
      dtype='object', length=195)

In [23]:
pd.set_option('display.max_columns', None)

In [61]:
agri_data["Item"].unique()


array(['Almonds, in shell',
       'Anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
       'Apples', 'Apricots', 'Asses', 'Barley',
       'Butter and ghee of sheep milk', 'Butter of cow milk',
       'Buttermilk, dry', 'Camels', 'Cantaloupes and other melons',
       'Cattle', 'Cattle fat, unrendered',
       'Cheese from milk of goats, fresh or processed',
       'Cheese from milk of sheep, fresh or processed', 'Chickens',
       'Cotton lint, ginned', 'Cotton seed', 'Cottonseed oil',
       'Edible offal of cattle, fresh, chilled or frozen',
       'Edible offal of goat, fresh, chilled or frozen',
       'Edible offal of sheep, fresh, chilled or frozen',
       'Edible offals of camels and other camelids, fresh, chilled or frozen',
       'Fat of camels', 'Figs', 'Game meat, fresh, chilled or frozen',
       'Goat fat, unrendered', 'Goats', 'Grapes',
       'Hen eggs in shell, fresh', 'Horses', 'Linseed', 'Maize (corn)',
       'Meat of camels, fresh or c

In [31]:
pd.set_option('display.max_rows', 100)